# Trabalho Final IA1

Começamos simplesmente importando tudo que usaremos no código, e carregando ambos os datasets de nosso folder.

In [ ]:
import pandas as pd
import numpy as np

# SKlearn imports
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import xgboost as xgb

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

Agora, vamos limpar as features. Precisamos retirar as colunas cujos valores são constantes, e as colunas que são duplicatas.

In [ ]:
# remove columns that are constant
remove = []
for col in train.columns:
    if train[col].std() == 0:
        remove.append(col)

train.drop(remove, axis=1, inplace=True)
test.drop(remove, axis=1, inplace=True)

# remove duplicated columns
remove = []
cols = train.columns
for i in range(len(cols)-1):
    v = train[cols[i]].values
    for j in range(i+1, len(cols)):
        if np.array_equal(v, train[cols[j]].values):
            remove.append(cols[j])

train.drop(remove, axis=1, inplace=True)
test.drop(remove, axis=1, inplace=True)

# test_id receives the ID column of the test CSV. It is then dropped from test, of course.
test_id = test.ID
test = test.drop(["ID"],axis=1)

# similarly, we drop both the target, and the ID from training set. Drop TARGET so as to make learning better.
X = train.drop(["TARGET","ID"],axis=1)
y = train.TARGET.values

# similar process to what was done in the entire course so far - split the training into two.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1729)

Nota-se que esta parte do código é extremamente simples. Retirar colunas constantes e colunas duplicatas é sempre algo que deva ser feito o mais cedo possível, para que não haja problema na hora de treinar os modelos com features desnecessárias. Faz parte do processo de analisar e aplicar feature engineering nos datasets.

Também, no final do código acima, simplesmente dropamos o ID da coluna de testes (desnecessário) e o salvamos em uma outra variável (para utilizar mais tarde, na submissão). E, de forma similar, dropamos não só o ID mas também o TARGET da nossa classe de treino (retirar o TARGET para podermos treinar, obviamente).

Por fim, dividimos o treino em dois, com 20% sendo utilizado para testar o fit do modelo, e 80% sendo realmente utilizado para treino. Isto é algo que foi feito ao longo de toda a disciplina, logo não há muito o que explicar.

A seguir, escolheremos quais features iremos utilizar dentre as restantes.

In [ ]:
# Then, we select the features we'll be utilizing to train our model.
clf = ExtraTreesClassifier(random_state=1729)
selector = clf.fit(X_train, y_train)

# selects the most important features we found through the extra tree classifier
fs = SelectFromModel(selector, prefit=True)
print(X_train.shape)
# reduces X_train and X_test to these features only.
X_train = fs.transform(X_train)
X_test = fs.transform(X_test)
test = fs.transform(test)
print(X_train.shape)

Nesta parte do código, utilizamos o ExtraTreeClassifier. Mas o que é o ExtraTreeClassifier?

O ExtraTreeClassifier gera e treina diversas árvores de decisões, em várias partições do arquivo de entrada, de tal forma a minimizar o overfit. Ao fitarmos esse modelo de ExtraTreesClassifier, estamos recebendo de volta uma floresta com as árvores criadas e treinadas. Ela é utilizada para treinar e descobrir quais features são mais importantes.

Por sua vez, o SelectFromModel é uma função para transformar nossas bases de dados. Como parametro, recebe o seletor de features que acabamos de criar. "Prefit" é "True" pois a árvore já foi previamente "fitada", ou treinada.

A função "transform", por fim, reduzirá todos os nossos datasets para conter apenas as features que julga necessárias.

Utilizando o print (X_train.shape), que nos diz qual a dimensão do nosso dataset, podemos perceber que antes de utilizarmos a função Transform no Dataset, haviam 306 colunas. Após o transform, restam apenas 36, que nosso ExtraTreesClassifier julgou útil. Não sabemos quais são estas features, porém, isto não é importante - dado que as features em si são "secretas" e os nomes são apenas nomes quaisquer, não há mérito em saber o que cada feature é a não ser curiosidade. Poderíamos plotar cada feature e descobrir qual o peso de cada sua importância, mas creio não ser necessário. Não é difícil, também, logo poderia ser implementado facilmente com um plot.

Com tudo isto feito, o que resta é utilizar Bagging, Stacking, e Ensemble para treinar diversos modelos e avaliar suas eficácias.

In [ ]:
# Train Models


# ALL SCORES FROM THE PRIVATE LEADERBOARD.


# XGB model. Private score: 0.819772

XGBmodel = xgb.XGBClassifier(n_estimators=110, nthread=-1, max_depth=4, seed=1729)
XGBmodel.fit(X_train, y_train, eval_metric="auc", verbose=False, eval_set=[(X_test, y_test)])
y_pred = XGBmodel.predict_proba(test)
submission = pd.DataFrame({"ID":test_id, "TARGET": y_pred[:, 1]})
submission.to_csv("submissionXGB.csv", index=False)

# KNN model. Private Score: 0.563689

KNNmodel = KNeighborsClassifier(n_neighbors=3, weights="distance")
KNNmodel.fit(X_train, y_train)
y_pred = KNNmodel.predict_proba(test)
submission = pd.DataFrame({"ID":test_id, "TARGET": y_pred[:, 1]})
submission.to_csv("submissionKNN.csv", index=False)

# Bagged KNN. Private Score: 0.661960.

baggedKNN = BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5, random_state=38)
baggedKNN.fit(X_train, y_train)
y_pred = baggedKNN.predict_proba(test)
submission = pd.DataFrame({"ID":test_id, "TARGET": y_pred[:, 1]})
submission.to_csv("submissionBaggedKNN.csv", index=False)

# MLPClassifier. Private Score: 0.535727

MLPCmodel = MLPClassifier(random_state=38)
MLPCmodel.fit(X_train, y_train)
y_pred = MLPCmodel.predict_proba(test)
submission = pd.DataFrame({"ID":test_id, "TARGET": y_pred[:, 1]})
submission.to_csv("submissionMLPC.csv", index=False)

# MLPclassifier (bagged). Private score: 0.701536

baggedMLPC = BaggingClassifier(MLPClassifier(random_state=38), max_samples=0.5, max_features=0.5, random_state=38)
baggedMLPC.fit(X_train, y_train)
y_pred = baggedMLPC.predict_proba(test)
submission = pd.DataFrame({"ID":test_id, "TARGET": y_pred[:, 1]})
submission.to_csv("submissionBaggedMLPC.csv", index=False)

# GBClassifier. Private Score: 0.818242

GBCmodel = GradientBoostingClassifier(random_state=38)
GBCmodel.fit(X_train, y_train)
y_pred = GBCmodel.predict_proba(test)
submission = pd.DataFrame({"ID":test_id, "TARGET": y_pred[:, 1]})
submission.to_csv("submissionGBC.csv", index=False)

# GBClassifier BAGGED. Private Score: 0.776595)

baggedGBC = BaggingClassifier(GradientBoostingClassifier(random_state=38), max_samples=0.5, max_features=0.5,
                             random_state=38)
baggedGBC.fit(X_train, y_train)
y_pred = baggedGBC.predict_proba(test)
submission = pd.DataFrame({"ID":test_id, "TARGET": y_pred[:, 1]})
submission.to_csv("submissionBaggedGBC.csv", index=False)

# DTClassifier. Private Score: 0.572142

DTCmodel = DecisionTreeClassifier(random_state=38, criterion="entropy")
DTCmodel.fit(X_train, y_train)
y_pred = DTCmodel.predict_proba(test)
submission = pd.DataFrame({"ID":test_id, "TARGET": y_pred[:, 1]})
submission.to_csv("submissionDTC.csv", index=False)

# DTClassifier BAGGED. Private Score: 0.701126

baggedDTC = BaggingClassifier(DecisionTreeClassifier(random_state=38), max_samples=0.5, max_features=0.5, random_state=38)
baggedDTC.fit(X_train, y_train)
y_pred = baggedDTC.predict_proba(test)
submission = pd.DataFrame({"ID":test_id, "TARGET": y_pred[:, 1]})
submission.to_csv("submissionBaggedDTC.csv", index=False)

Acima vemos o stacking e o bagging de modelos, mas ainda sem um ensemble para todos. O bagging utilizado aqui é apenas para a criação de diversos modelos, aumentando sua eficácia agregando os resultados para minimizar a variância. 

Vemos que, de forma geral, o Bagging melhora a vasta maioria de todos os modelos testados, tendo melhoria de até 17% (como no caso da rede neural, MLPClassifier, que utiliza multilayer perceptrons como já visto em sala). A única exceção a esta regra é o GradientBoostClassifier, que piora levemente com a utilização do Bagging.

Por sua vez, podemos perceber que o GBClassifier está bem próximo ao XGBoost, enquanto DecisionTreeClassifier, KNearestNeighbors, e o próprio MLPClassifier (sem bagging) estão significativamente piores.

Em relação aos valores com a utilização de Bagging, GBClassifier continua com o maior score dentre os baggings utilizados, porém é o único com uma piora - todos os outros modelos obtem uma melhoria, com o MLPClassifer e o DTClassifier estando bem próximos. KNN continua, porém, significativamente atrás. 

De forma geral, a ordem dos modelos treinados individualmente, em eficiência, segue desta forma:

1. XGBoost - 0.819772
2. GBClassifier - 0.818242
3. GBClassifier com Bagging - 0.776595
4. MLPClassifier com Bagging - 0.701536
5. DTClassifier com Bagging - 0.701126
6. KNN com Bagging - 0.661960
7. DTClassifier - 0.572142
8. MLPClassifier - 0.535727
9. KNN - 0.563689

Nota-se que a melhor melhora com a utilização de bagging é vista no MLPClassifier.

Para analisar estes modelos, não mudamos os parâmetros destes. Como o objetivo era analisar as diferenças de baggings, não havia porque mexer nos parâmetros individuais de cada modelo, dado que a ênfase deste trabalho está em analisar a diferença do modelo por si só e com a utilização de bagging, e não sua eficácia com diversos parâmetros.

Todos os scores aqui colocados são os resultados obtidos na Leaderboard Privada do kaggle.

Por fim, temos então a utilização do Ensemble.

In [ ]:
# Ensembling everything (VOTING CLASSIFIER....)
# --------------------------------------------------------------------
# first lets try ensembling with no baggings... Private Score: 0.781657

ensembleVC = VotingClassifier(estimators=[('xgb', XGBmodel), ('knn', KNNmodel), ('mlpc', MLPCmodel), ('gbc', GBCmodel)
                                           ,('dtc', DTCmodel)], voting="soft")
ensembleVC.fit(X_test, y_test)
y_pred = ensembleVC.predict_proba(test)
submission = pd.DataFrame({"ID":test_id, "TARGET": y_pred[:, 1]})
submission.to_csv("submissionensembleNoBags.csv", index=False)

# ---------------------------------------------------------------------

# now let us ensemble with only baggings... private Score: 0.769105

ensembleVC = VotingClassifier(estimators=[('xgb', XGBmodel), ('knnbag', baggedKNN), ('mlpcbag', baggedMLPC), ('gbcbag', baggedGBC),
                                          ('dcbag', baggedDTC)], voting="soft")
ensembleVC.fit(X_test, y_test)
y_pred = ensembleVC.predict_proba(test)
submission = pd.DataFrame({"ID":test_id, "TARGET": y_pred[:, 1]})
submission.to_csv("submissionensembleOnlyBags.csv", index=False)

# ---------------------------------------------------------------------

# now, ensembling EVERYTHING... Private Score: 0.763175

ensembleVC = VotingClassifier(estimators=[('xgb', XGBmodel), ('knn', KNNmodel), ('knnbag', baggedKNN),
                                         ('mlpc', MLPCmodel), ('mlpcbag', baggedMLPC), ('gbc', GBCmodel),
                                         ('gbcbag', baggedGBC), ('dtc', DTCmodel), ('dtcbag', baggedDTC)],
                              voting="soft");
ensembleVC.fit(X_test, y_test)
y_pred = ensembleVC.predict_proba(test)
submission = pd.DataFrame({"ID":test_id, "TARGET": y_pred[:, 1]})
submission.to_csv("submissionensembleALL.csv", index=False)


O ensemble utilizado foi o VotingClassifier, um ensemble simple de voto de maioria. Ele fita um clone dos modelos (que resultam no mesmo que anteriormente, dado que todos estão utilizando sementes). Cada modelo pode ter um peso, porém aqui, não colocamos nenhum peso em nenhum modelo - isto se deve ao fato de que queremos mostrar que este Ensemble funciona melhor com modelos que possuem uma discrepância significativa, como visto no primeiro resultado (onde utilizamos os modelos sem os baggings, que aumentam de forma visível a eficácia de cada modelo).

O parâmetro "voting=soft" é utilizado para poder utilizar a função "predict_proba", que nos retorna a probabilidade do nosso TARGET, mantendo o sistema consistente com tudo utilizado até então.